<h1><center>POS tagging using modified Viterbi</center></h1>

<div style="text-align: right"> Submitted By: Kashif Sami

## Data Preparation

In [1]:
#Importing libraries
import nltk
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import random
from collections import Counter

In [2]:
#Configure notebook

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Set Parameters for Displaying data
pd.options.display.max_info_columns = 300
pd.set_option('display.max_columns', 120)
pd.options.display.max_rows = 300

np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:.4f}'.format

# #InteractiveShell.ast_node_interactivity = "all"
sns.set_style("whitegrid")
%matplotlib inline


# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [3]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents(tagset='universal')) #tagset='universal'
len(wsj)
wsj[:2]

3914

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')]]

In [4]:
#Train-test split
random.seed(100)
train_set, test_set = train_test_split(wsj,test_size=0.05)
len(train_set)
len(test_set)

3718

196

In [5]:
#Build List of train tagged tokens/words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)
train_tagged_words[:10]

95688

[('Instead', 'ADV'),
 (',', '.'),
 ('the', 'DET'),
 ('companies', 'NOUN'),
 ('will', 'VERB'),
 ('leave', 'VERB'),
 ('it', 'PRON'),
 ('up', 'ADP'),
 ('to', 'PRT'),
 ('the', 'DET')]

In [6]:
#Build list of tokens only
tokens = [pair[0] for pair in train_tagged_words]
len(tokens)
tokens[:10]

95451

['But',
 '*',
 'maintaining',
 'U.S.',
 'influence',
 'will',
 'be',
 'difficult',
 'in',
 'the']

In [7]:
#Vocabulary or unique tokens
V = set(tokens)
len(V)

12028

In [8]:
#No. of unique POS tags
T = set([pair[1] for pair in train_tagged_words])
len(T)
T

12

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

>**Observation**: Clearly, the POS tags are limited to the 'universal' set

In [9]:
#Prepare test data set

# random.seed(100)

# # choose random 5 sents
# rndom = [random.randint(1,len(test_set)) for x in range(5)]
# rndom[:10]

# list of sents
# test_run = [test_set[i] for i in rndom]
# test_run[:1]
# test_set
# test_run

# list of tagged test words
test_tagged_words = [tup for sent in test_set for tup in sent]
test_tagged_words[:10]

test_tagged_words

# list of untagged test words
test_no_tag = [tup[0] for sent in test_set for tup in sent]
test_no_tag[:10]
# test_run

[('Diamond', 'NOUN'),
 ('Creek', 'NOUN'),
 ('1985', 'NUM'),
 ('Lake', 'NOUN'),
 ('Vineyard', 'NOUN'),
 ('Cabernet', 'NOUN'),
 ('weighed', 'VERB'),
 ('in', 'PRT'),
 ('this', 'DET'),
 ('fall', 'NOUN')]

[('Diamond', 'NOUN'),
 ('Creek', 'NOUN'),
 ('1985', 'NUM'),
 ('Lake', 'NOUN'),
 ('Vineyard', 'NOUN'),
 ('Cabernet', 'NOUN'),
 ('weighed', 'VERB'),
 ('in', 'PRT'),
 ('this', 'DET'),
 ('fall', 'NOUN'),
 ('with', 'ADP'),
 ('a', 'DET'),
 ('sticker', 'NOUN'),
 ('price', 'NOUN'),
 ('of', 'ADP'),
 ('$', '.'),
 ('100', 'NUM'),
 ('*U*', 'X'),
 ('a', 'DET'),
 ('bottle', 'NOUN'),
 ('.', '.'),
 ('Its', 'PRON'),
 ('new', 'ADJ'),
 ('products', 'NOUN'),
 ('and', 'CONJ'),
 ('trading', 'NOUN'),
 ('techniques', 'NOUN'),
 ('have', 'VERB'),
 ('been', 'VERB'),
 ('highly', 'ADV'),
 ('profitable', 'ADJ'),
 ('.', '.'),
 ('Fees', 'NOUN'),
 ('1', 'NUM'),
 ('3\\/4', 'NUM'),
 ('.', '.'),
 ('The', 'DET'),
 ('aerospace', 'NOUN'),
 (',', '.'),
 ('automotive', 'ADJ'),
 ('supply', 'NOUN'),
 (',', '.'),
 ('electronics', 'NOUN'),
 ('and', 'CONJ'),
 ('printing-press', 'NOUN'),
 ('concern', 'NOUN'),
 ('also', 'ADV'),
 ('indicated', 'VERB'),
 ('that', 'ADP'),
 ('the', 'DET'),
 ('first', 'ADJ'),
 ('half', 'DET'),
 ('of', 'A

['Diamond',
 'Creek',
 '1985',
 'Lake',
 'Vineyard',
 'Cabernet',
 'weighed',
 'in',
 'this',
 'fall']

### Build the vanilla Viterbi based POS tagger

#### Emission Probabilities

In [10]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [11]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag] #List of words that have the 'tag' POS tag
    count_tag = len(tag_list) 
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word] #How many of them include 'word'
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [12]:
# Test outcome for the word 'book'
print("\n", "book")
print(word_given_tag('book', 'NOUN'))
print(word_given_tag('book', 'VERB'))


 book
(7, 27288)
(1, 12900)


#### Transition Probabilities

In [13]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]      #Get all tags in training data including duplicates
    count_t1 = len([t for t in tags if t==t1])  #No. of times the tag T1 shows up in training data
    count_t2_t1 = 0
    
    #No. of times t1 is followed by t2
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1                   
            
    return (count_t2_t1, count_t1)

In [14]:
# Test outcome for words at start of sentence
print(t2_given_t1('VERB', '.'))
print(t2_given_t1('NOUN', '.'))
print(t2_given_t1('PRON', '.'))
print(t2_given_t1('DET', '.'))

(978, 11036)
(2420, 11036)
(741, 11036)
(1938, 11036)


In [15]:
#Create transition probilities matrix to be referenced later
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
        
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [16]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            #If word is at start of sentence
            if key == 0:
                transition_p = tags_df.loc['.', tag]
                
            #If word is NOT at start of sentence
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        
        # get state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        
    return list(zip(words, state)) #Return tagged words



#### Evaluate tagging accuracy for vanilla Viterbi

In [17]:
# tagging the test sentences
test_tags_pred = Viterbi(test_no_tag)

In [18]:
# accuracy
correct_tags = [i for i, j in zip(test_tags_pred, test_tagged_words) if i == j] 
accuracy = len(correct_tags)/len(test_tags_pred)
accuracy

0.9066028708133971

**Observation**: The baseline accuracy using vanilla Viterbi on the validation set (5% of tagged data) is 

## Solve the problem of unknown words

### Approach 1
For unknown words, the tag defaults to the first tag in the list of universal tags because the emission probabilities of ALL tags would be zero. So, **whenever a word is unknown i.e. emission probability = 0, let's only consider the transition probabilities for predicting the tag**.

In [19]:
# Viterbi Heuristic
def Viterbi2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        
        #initialise list of probability column for a given observation
        p = []
        tp = []
        for tag in T:
            #If word is at start of sentence
            if key == 0:
                transition_p = tags_df.loc['.', tag]
                
            #If word is NOT at start of sentence
            else:
                transition_p = tags_df.loc[state[-1], tag]
              
            tp.append(transition_p)
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
        
        pmax = max(p)
        
        #If word is uknown, get max transition probability
        if pmax==0:
            pmax = max(tp)
#             print("word:",word)
            state_max = T[tp.index(pmax)] # getting state for which transition probability is maximum
#             print("state_max",state_max)

        #If word is known use vanilla Viterbi algorithm
        else:
            state_max = T[p.index(pmax)] # getting state for which probability is maximum

        state.append(state_max)
    return list(zip(words, state)) #Return tagged words



In [20]:
# tagging the test sentences
test_tags_pred2 = Viterbi2(test_no_tag)

### Approach 2
**Let's look for patterns in the unknown words and try to build regex rules targeted to tag them correctly.**

In [21]:
#Most common tag?
Counter([pair[1] for pair in train_tagged_words]).most_common(5)

[('NOUN', 27288), ('VERB', 12900), ('.', 11036), ('ADP', 9394), ('DET', 8295)]

>**Observation**: Noun is the most common tag

In [22]:
# Get list of unknown words in the train set 
def get_unknowns(words, train_bag = train_tagged_words):
    unknowns = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        
        # compute emission probabilities for word-tag pairs
        for tag in T:
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            p.append(emission_p)

        pmax = max(p)
        
        #If the word is unknown, collect list of words
        if pmax==0:
            unknowns.append(word)
            
    return unknowns #Return list of unknowns words



In [23]:
#Print list of unknown words and corresponding tags
for word in get_unknowns(words=test_no_tag):
    if(dict(test_tagged_words)[word]):
        print(word,dict(test_tagged_words)[word])

Vineyard NOUN
weighed VERB
sticker NOUN
printing-press NOUN
rough ADJ
Generalized NOUN
Preferences NOUN
46.1 NUM
1.85 NUM
1.85 NUM
cereal NOUN
favors VERB
accompany VERB
pain NOUN
unanimous ADJ
184 NUM
*-30 X
Planters NOUN
Memphis NOUN
Tenn. NOUN
Edge NOUN
thirtysomething NOUN
crowd NOUN
*T*-197 X
1991-2000 NUM
trip NOUN
mindful ADJ
stuck VERB
setting VERB
crunch NOUN
crunch NOUN
crunch NOUN
bang NOUN
bang NOUN
bang NOUN
obvious ADJ
exit NOUN
prayer NOUN
Regarded VERB
Vineyard NOUN
Mannix NOUN
Deposits-a NOUN
6.21 NUM
Cole NOUN
Jackson NOUN
Rita NOUN
Rae NOUN
Cross NOUN
Denver NOUN
Meinders NOUN
five-day ADJ
eight-month ADJ
Baton NOUN
Rouge NOUN
La. NOUN
one-month ADJ
Karl NOUN
Grant NOUN
Hale NOUN
Midvale NOUN
Utah NOUN
Clinton NOUN
Hayne NOUN
one-week ADJ
Coconut NOUN
250,000 NUM
Merrick NOUN
Aurora NOUN
Baton NOUN
Rouge NOUN
Pace NOUN
90-day ADJ
Brian NOUN
Pitcher NOUN
Russo NOUN
Bridgeville NOUN
15-day ADJ
Orville NOUN
Leroy NOUN
Sandberg NOUN
Aurora NOUN
Marchese NOUN
Eric NOUN
Mo

>**Observation**: There are several patterns in these unknown word-tag pairs that I've captured below using regular expression patterns. They may not be perfect such as some words ending in 'ing' are adverbs but I've tried to capture the most commonly associated tag for each pattern

In [24]:
# Specify patterns based on observing unknown words plus some other rules
patterns = [
    ('.[0-9]{1,}\.{0,}-{0,}[0-9]{0,}','NUM'),      # number
    (r'.*ing$', 'VERB'),                           # words ending in ing are often gerund
    (r'.*ed$', 'VERB'),                            # words ending in ed are often past tense verbs
    (r'.*ize$', 'VERB'),                           # words ending in ize are often verbs
    (r'.*ied$', 'VERB'),                           # words ending in ied are often past tense verbs
    (r'.*iest$', 'ADJ'),                           # words ending in iest are often superlative adjectives
    (r'.*ion$', 'NOUN'),                           # words ending in ion are often nouns
    (r'.*ies$', 'NOUN'),                           # words ending in ies are often nouns
    (r'.*\'s$', 'NOUN'),                           # words ending in 's  are often posessive nouns
    (r'.*s$', 'NOUN'),                             # words ending in s,  are often plural nouns
    (r'.*', 'NOUN'),                               # Default to Noun because it is the most common tag
]

In [25]:
# Create rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [26]:
# Modify Viterbi algorithm to tag unknown words with regex based rule-based tagger
def Viterbi3(words, train_bag = train_tagged_words,tagger = rule_based_tagger):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):

        #initialise list of probability column for a given observation
        p = []
#         tp = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
              
            # tp.append(transition_p)
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
#           state_probability = transition_p if emission_p==0 else emission_p * transition_p    
            p.append(state_probability)
        
    
        pmax = max(p)
        
        #If word is unknown, tag with rule based tagger
        if pmax==0:
            state_max = tagger.tag([word])[0][1]
#             print("word:",word)
#             print("state_max",state_max)
        
        #If word is known, tag using vanilla Viterbi algorithm
        else:
            state_max = T[p.index(pmax)] # getting state for which probability is maximum
#             print("state_max",state_max)

        
        state.append(state_max)
    return list(zip(words, state))



In [27]:
# tagging the test sentences
test_tags_pred3 = Viterbi3(test_no_tag)

## Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [28]:
#accuracy for vanilla Viterbi
correct_tags = [i for i, j in zip(test_tags_pred, test_tagged_words) if i == j] 
accuracy = len(correct_tags)/len(test_tags_pred)
accuracy

0.9066028708133971

In [29]:
# accuracy for modification 1 
correct_tags = [i for i, j in zip(test_tags_pred2, test_tagged_words) if i == j] 
accuracy = len(correct_tags)/len(test_tags_pred2)
accuracy

0.938755980861244

In [30]:
# accuracy for modification 2
correct_tags = [i for i, j in zip(test_tags_pred3, test_tagged_words) if i == j] 
accuracy = len(correct_tags)/len(test_tags_pred3)
accuracy

0.9529186602870814

>**Observation**: Clearly, the tagging accuracy on the validation (test data obtained from test-train split) improves from about 90% to about 94% and 95% using the modified Viterbi algorithms.

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

Let's tag the test sentences/words given in the file using all 3 approaches and analyze outcomes.

In [31]:
#Let's load the test sentences
test_sent_df = pd.read_table("Test_sentences.txt",header=None,delimiter="\n")
test_sent_df

,0
0,Android is a mobile operating system developed...
1,Android has been the best-selling OS worldwide...
2,Google and Twitter made a deal in 2015 that ga...
3,Twitter is an online news and social networkin...
4,"Before entering politics, Donald Trump was a d..."
5,The 2018 FIFA World Cup is the 21st FIFA World...
6,This is the first World Cup to be held in East...
7,Show me the cheapest round trips from Dallas t...
8,I would like to see flights from Denver to Phi...
9,Show me the price of the flights leaving Atlan...


In [32]:
#Let's tokenize the test sentences
test_tokens = [token for sent in test_sent_df[0] for token in nltk.word_tokenize(sent)]
test_tokens

['Android',
 'is',
 'a',
 'mobile',
 'operating',
 'system',
 'developed',
 'by',
 'Google',
 '.',
 'Android',
 'has',
 'been',
 'the',
 'best-selling',
 'OS',
 'worldwide',
 'on',
 'smartphones',
 'since',
 '2011',
 'and',
 'on',
 'tablets',
 'since',
 '2013',
 '.',
 'Google',
 'and',
 'Twitter',
 'made',
 'a',
 'deal',
 'in',
 '2015',
 'that',
 'gave',
 'Google',
 'access',
 'to',
 'Twitter',
 "'s",
 'firehose',
 '.',
 'Twitter',
 'is',
 'an',
 'online',
 'news',
 'and',
 'social',
 'networking',
 'service',
 'on',
 'which',
 'users',
 'post',
 'and',
 'interact',
 'with',
 'messages',
 'known',
 'as',
 'tweets',
 '.',
 'Before',
 'entering',
 'politics',
 ',',
 'Donald',
 'Trump',
 'was',
 'a',
 'domineering',
 'businessman',
 'and',
 'a',
 'television',
 'personality',
 '.',
 'The',
 '2018',
 'FIFA',
 'World',
 'Cup',
 'is',
 'the',
 '21st',
 'FIFA',
 'World',
 'Cup',
 ',',
 'an',
 'international',
 'football',
 'tournament',
 'contested',
 'once',
 'every',
 'four',
 'years',
 '.'

In [33]:
#Let's set aside the words in test sentences that are NOT present in the training set
unknown_test = []
for word in get_unknowns(words=test_tokens):
    unknown_test.append(word)
    
len(unknown_test)
unknown_test

38

['Android',
 'Google',
 'Android',
 'OS',
 'worldwide',
 'smartphones',
 '2011',
 '2013',
 'Google',
 'Twitter',
 '2015',
 'Google',
 'Twitter',
 'firehose',
 'Twitter',
 'online',
 'interact',
 'messages',
 'tweets',
 'domineering',
 'personality',
 '2018',
 'FIFA',
 'Cup',
 '21st',
 'FIFA',
 'Cup',
 'tournament',
 'contested',
 'Cup',
 '11th',
 'trips',
 'Denver',
 'arriving',
 'NASA',
 'invited',
 'ICESAT-2',
 'Satellite']

In [34]:
#Let's build a dataframe of unknown words for comparison of tagging results
unknown_df = pd.DataFrame(unknown_test)
unknown_df = unknown_df.rename(columns={0: "tokens"})
unknown_df.head()

,tokens
0,Android
1,Google
2,Android
3,OS
4,worldwide


>**Observation**: There are 36 unknown words

In [35]:
# Tag the test words with vanilla Viterbi
test_pred = Viterbi(test_tokens)

In [36]:
unknown_df['viterbi_vanilla_tags'] = unknown_df['tokens'].map(lambda x:dict(test_pred).get(x))

In [37]:
# Tag the test words with approach 1 (use transition probabilities for unknown words)
test_pred2 = Viterbi2(test_tokens)
test_pred2

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'DET'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'DET'),
 ('since', 'ADP'),
 ('2011', 'DET'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'DET'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'DET'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'X'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'VERB'),
 ("'s", 'PRT'),
 ('firehose', 'VERB'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('servic

In [38]:
unknown_df['viterbi2_tags'] = unknown_df['tokens'].map(lambda x:dict(test_pred2).get(x))

In [39]:
# Tag the test words with approach 1 (use rule-based tagger for unknown words)
test_pred3 = Viterbi3(test_tokens)

In [40]:
unknown_df['viterbi3_tags'] = unknown_df['tokens'].map(lambda x:dict(test_pred3).get(x))

In [41]:
unknown_df

,tokens,viterbi_vanilla_tags,viterbi2_tags,viterbi3_tags
0,Android,NUM,NOUN,NOUN
1,Google,NUM,X,NOUN
2,Android,NUM,NOUN,NOUN
3,OS,NUM,NOUN,NOUN
4,worldwide,NUM,NOUN,NOUN
5,smartphones,NUM,DET,NOUN
6,2011,NUM,DET,NUM
7,2013,NUM,DET,NUM
8,Google,NUM,X,NOUN
9,Twitter,NUM,NOUN,NOUN


**Observations**: As can be seen above, the vanilla Viterbi algorithm defaults to the first tag whereas the modified viterbi functions Viterbi1 (transition probabilities for unknown words) and Viterbi2 (rule-based tagger for unknown words) perform much better. Some examples below:

- Numbers **2011,2013 and 2018** are taggeted correctly as number by Viterbi2
- Words like **arriving & invited** are correctly tagged as Verb by Viterbi2 because of the -ing and -ed ending rule. 
- Words like **messages & tweets** are correctly tagged as Noune by Viterbi2 because of the -s ending rule. 
- Words like **Android, FIFA, Cup, Twitter & Satellite** are correctly tagged as Noun by Viterbi1. Viterbi2 is right as well but only because it defaults to Noune when no rule is applicable.
- The word **domineering** is incorrectly tagged as Verb by Viterbi2 because of the -ing ending rule. It's suppose to be an Adjective.

**Conclusion**: The modifications to Viterbi certainly improved the tagging accuracy of unknown words. The rule-based tagger fails whenever there's an exception to its rigid rules whereas the transition probabilities learnt from train data set performs better. 